# Python II - Group Assignment - synthetic credit score
## An approach to detecting and understanding machine learning biases using classes

In [1]:
# Creating our classes


class bias_detection:    
    
    def create_labels(X_test,sensitive):
        sensitive_label = {}
        for i in set(X_test[sensitive]):
            text = "Num_Children" +" "+ str(i)+": "
            label = input(text)
            sensitive_label[i]=label
        return(sensitive_label)

    def representation(X_test,y_test,sensitive,labels,predictions):
        full_table = X_test.copy()
        sens_df = {}

        for i in labels:
            full_table['p'] = predictions
            full_table['t'] = y_test
            sens_df[labels[i]] = full_table[full_table[sensitive]==i]

        contigency_p = pd.crosstab(full_table[sensitive], full_table['t']) 
        cp, pp, dofp, expectedp = chi2_contingency(contigency_p) 
        contigency_pct_p = pd.crosstab(full_table[sensitive], full_table['t'], normalize='index')

        sens_rep = {}
        for i in labels:
            sens_rep[labels[i]] = (X_test[sensitive].value_counts()/X_test[sensitive].value_counts().sum())[i]

        labl_rep = {}
        for i in labels:
            labl_rep[str(i)] = (y_test.value_counts()/y_test.value_counts().sum())[i]


        fig = make_subplots(rows=1, cols=2)

        for i in labels:
            fig.add_trace(go.Bar(
            showlegend=False,
            x = [labels[i]],
            y= [sens_rep[labels[i]]]),row=1,col=1)

            fig.add_trace(go.Bar(
            showlegend=False,
            x = [str(i)],
            y= [labl_rep[str(i)]],
            marker_color=['orange','blue'][i]),row=1,col=2)

        c, p, dof, expected = chi2_contingency(contigency_p)
        cont_table = (tabulate(contigency_pct_p.T, headers=labels.values(), tablefmt='fancy_grid'))

        return cont_table, sens_df, fig, p

    def ability(sens_df,labels):
        sens_conf = {}
        for i in labels:
            sens_conf[labels[i]] = confusion_matrix(list(sens_df[labels[i]]['t']), list(sens_df[labels[i]]['p']), labels=[0,1]).ravel()

        true_positive_rate = {}
        false_positive_rate = {}
        true_negative_rate = {}
        false_negative_rate = {}

        for i in labels:
            true_positive_rate[labels[i]] = (sens_conf[labels[i]][3]/(sens_conf[labels[i]][3]+sens_conf[labels[i]][2]))
            false_positive_rate[labels[i]] = (sens_conf[labels[i]][1]/(sens_conf[labels[i]][1]+sens_conf[labels[i]][0]))
            true_negative_rate[labels[i]] = 1 - false_positive_rate[labels[i]]
            false_negative_rate[labels[i]] = 1 - true_positive_rate[labels[i]]

        return(true_positive_rate,false_positive_rate,true_negative_rate,false_negative_rate)


    def ability_plots(labels,TPR,FPR,TNR,FNR):
        fig = make_subplots(rows=2, cols=2, 
                            subplot_titles=("True Positive Rate", "False Positive Rate", "True Negative Rate", "False Negative Rate"))

        x_axis = list(labels.values())
        fig.add_trace(
            go.Bar(x = x_axis, y=list(TPR.values())),
            row=1, col=1
        )

        fig.add_trace(
            go.Bar(x = x_axis, y=list(FPR.values())),
            row=1, col=2
        )

        fig.add_trace(
            go.Bar(x = x_axis, y=list(TNR.values())),
            row=2, col=1
        )

        fig.add_trace(
            go.Bar(x = x_axis, y=list(FNR.values())),
            row=2, col=2
        )

        fig.update_layout(showlegend=False,height=600, width=800, title_text="Ability Disparities")
        fig.show()


    def ability_metrics(TPR,FPR,TNR,FNR):
        TPR_p = chisquare(list(np.array(list(TPR.values()))*100))[1]
        FPR_p = chisquare(list(np.array(list(FPR.values()))*100))[1]
        TNR_p = chisquare(list(np.array(list(TNR.values()))*100))[1]
        FNR_p = chisquare(list(np.array(list(FNR.values()))*100))[1]

        if TPR_p <= 0.01:
            print("*** Reject H0: Significant True Positive Disparity with p=",TPR_p)
        elif TPR_p <= 0.05:
            print("** Reject H0: Significant True Positive Disparity with p=",TPR_p)
        elif TPR_p <= 0.1:
            print("*  Reject H0: Significant True Positive Disparity with p=",TPR_p)
        else:
            print("Accept H0: True Positive Disparity Not Detected. p=",TPR_p)

        if FPR_p <= 0.01:
            print("*** Reject H0: Significant False Positive Disparity with p=",FPR_p)
        elif FPR_p <= 0.05:
            print("** Reject H0: Significant False Positive Disparity with p=",FPR_p)
        elif FPR_p <= 0.1:
            print("*  Reject H0: Significant False Positive Disparity with p=",FPR_p)
        else:
            print("Accept H0: False Positive Disparity Not Detected. p=",FPR_p)

        if TNR_p <= 0.01:
            print("*** Reject H0: Significant True Negative Disparity with p=",TNR_p)
        elif TNR_p <= 0.05:
            print("** Reject H0: Significant True Negative Disparity with p=",TNR_p)
        elif TNR_p <= 0.1:
            print("*  Reject H0: Significant True Negative Disparity with p=",TNR_p)
        else:
            print("Accept H0: True Negative Disparity Not Detected. p=",TNR_p)

        if FNR_p <= 0.01:
            print("*** Reject H0: Significant False Negative Disparity with p=",FNR_p)
        elif FNR_p <= 0.05:
            print("** Reject H0: Significant False Negative Disparity with p=",FNR_p)
        elif FNR_p <= 0.1:
            print("*  Reject H0: Significant False Negative Disparity with p=",FNR_p)
        else:
            print("Accept H0: False Negative Disparity Not Detected. p=",FNR_p)


    def predictive(labels,sens_df):
        precision_dic = {}

        for i in labels:
            precision_dic[labels[i]] = precision_score(sens_df[labels[i]]['t'],sens_df[labels[i]]['p'])

        fig = go.Figure([go.Bar(x=list(labels.values()), y=list(precision_dic.values()))])

        pred_p = chisquare(list(np.array(list(precision_dic.values()))*100))[1]

        return(precision_dic,fig,pred_p)


    def identify_bias(model,X_test,y_test,sensitive,labels):
        predictions = model.predict(X_test)
        cont_table,sens_df,rep_fig,rep_p = representation(X_test,y_test,sensitive,labels,predictions)

        print("REPRESENTATION")
        rep_fig.show()

        print(cont_table,'\n')

        if rep_p <= 0.01:
            print("*** Reject H0: Significant Relation Between",sensitive,"and Target with p=",rep_p)
        elif rep_p <= 0.05:
            print("** Reject H0: Significant Relation Between",sensitive,"and Target with p=",rep_p)
        elif rep_p <= 0.1:
            print("* Reject H0: Significant Relation Between",sensitive,"and Target with p=",rep_p)
        else:
            print("Accept H0: No Significant Relation Between",sensitive,"and Target Detected. p=",rep_p)

        TPR, FPR, TNR, FNR = ability(sens_df,labels)
        print("\n\nABILITY")
        ability_plots(labels,TPR,FPR,TNR,FNR)
        ability_metrics(TPR,FPR,TNR,FNR)


        precision_dic, pred_fig, pred_p = predictive(labels,sens_df)
        print("\n\nPREDICTIVE")
        pred_fig.show()

        if pred_p <= 0.01:
            print("*** Reject H0: Significant Predictive Disparity with p=",pred_p)
        elif pred_p <= 0.05:
            print("** Reject H0: Significant Predictive Disparity with p=",pred_p)
        elif pred_p <= 0.1:
            print("* Reject H0: Significant Predictive Disparity with p=",pred_p)
        else:
            print("Accept H0: No Significant Predictive Disparity. p=",pred_p)

    def understand_shap(X_test,y_test,model,labels,sensitive,affected_group,affected_target):
        import shap
        explainer = shap.Explainer(model)

        full_table = X_test.copy()
        full_table['t'] = y_test
        full_table['p'] = model.predict(X_test)
        full_table

        shap_values = explainer(X_test)
        sens_glob_coh = np.where(X_test[sensitive]==list(labels.keys())[0],labels[0],labels[1])

        misclass = full_table[full_table.t != full_table.p]
        affected_class = misclass[(misclass[sensitive] == affected_group) & (misclass.p == affected_target)]
        shap_values2 = explainer(affected_class.drop(['t','p'],axis=1))
        #sens_mis_coh = np.where(affected_class[sensitive]==list(labels.keys())[0],labels[0],labels[1])


        figure,axes = plt.subplots(nrows=2, ncols=2,figsize=(20,10))
        plt.subplots_adjust(right=1.4,wspace=1)

        print("Model Importance Comparison")
        plt.subplot(1, 2, 1) # row 1, col 2 index 1
        shap.plots.bar(shap_values.cohorts(sens_glob_coh).abs.mean(0),show=False)
        plt.subplot(1, 2, 2) # row 1, col 2 index 1
        shap_values2 = explainer(affected_class.drop(['t','p'],axis=1))
        shap.plots.bar(shap_values2)
        #shap.plots.bar(shap_values2)

        full_table['t'] = y_test
        full_table['p'] = model.predict(X_test)
        #full_table=full_table[['checking_account','credit_amount','duration','sex','t','p']]

        misclass = full_table[full_table.t != full_table.p]
        affected_class = misclass[(misclass[sensitive] == affected_group) & (misclass.p == affected_target)]

        truclass = full_table[full_table.t == full_table.p]
        tru_class = truclass[(truclass[sensitive] == affected_group) & (truclass.t == affected_target)]

        x_axis = list(affected_class.drop(['t','p',sensitive],axis=1).columns)
        affect_character = list((affected_class.drop(['t','p',sensitive],axis=1).mean()-tru_class.drop(['t','p',sensitive],axis=1).mean())/affected_class.drop(['t','p',sensitive],axis=1).mean())

        #plt.figsize([10,10])
        #plt.bar(x_axis,affect_character)

        fig = go.Figure([go.Bar(x=x_axis, y=affect_character)])

        print("Affected Attribute Comparison")
        print("Average Comparison to True Class Members")
        fig.show()

        misclass = full_table[full_table.t != full_table.p]
        affected_class = misclass[(misclass[sensitive] == affected_group) & (misclass.p == affected_target)]

        #truclass = full_table[full_table.t == full_table.p]
        tru_class = full_table[(full_table[sensitive] == affected_group) & (full_table.p == affected_target)]

        x_axis = list(affected_class.drop(['t','p',sensitive],axis=1).columns)
        affect_character = list((affected_class.drop(['t','p',sensitive],axis=1).mean()-full_table.drop(['t','p',sensitive],axis=1).mean())/affected_class.drop(['t','p',sensitive],axis=1).mean())

        #plt.figsize([10,10])
        #plt.bar(x_axis,affect_character)

        fig = go.Figure([go.Bar(x=x_axis, y=affect_character)])
        print("Average Comparison to All Members")
        fig.show()

        print("Random Affected Decision Process")
        explainer = shap.Explainer(model)
        shap.plots.waterfall(explainer(affected_class.drop(['t','p'],axis=1))[randrange(0, len(affected_class))],show=False)

In [2]:
# Importing our Libraries

import matplotlib.pyplot as plt
from random import randrange
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import dalex as dx
import pandas as pd
from scipy.stats import chi2_contingency
from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from scipy.stats import chisquare
from sklearn.metrics import precision_score
import fairdetect_functions as fd
from __future__ import print_function
######################################################################
import sqlite3

In [3]:
# Loading our DataFrame
data = pd.read_csv('synthetic_credit_card_approval.csv')
data

Num_Children  Group  Income  Own_Car  Own_Housing  Target
0                  4      0   36151        0            0       0
1                  0      0   36095        1            0       0
2                  2      1   62110        1            1       0
3                  0      1   73644        1            0       0
4                  3      0   99146        0            0       1
...              ...    ...     ...      ...          ...     ...
499995             1      0   43299        1            1       0
499996             8      1   75956        1            1       1
499997             3      0   66476        0            0       0
499998             2      1   74524        0            0       0
499999             4      0   31129        1            0       0

[500000 rows x 6 columns]

In [4]:
data.describe()

Num_Children         Group         Income        Own_Car  \
count  500000.000000  500000.00000  500000.000000  500000.000000   
mean        2.000346       0.50065   72507.446898       0.700930   
std         1.410574       0.50000   22960.209440       0.457851   
min         0.000000       0.00000   30000.000000       0.000000   
25%         1.000000       0.00000   53321.000000       0.000000   
50%         2.000000       1.00000   72060.000000       1.000000   
75%         3.000000       1.00000   90670.250000       1.000000   
max        11.000000       1.00000  119999.000000       1.000000   

         Own_Housing         Target  
count  500000.000000  500000.000000  
mean        0.401612       0.386626  
std         0.490225       0.486977  
min         0.000000       0.000000  
25%         0.000000       0.000000  
50%         0.000000       0.000000  
75%         1.000000       1.000000  
max         1.000000       1.000000

In [6]:
############# ADDED SQL #################
##### storing raw dF ####################
conn=sqlite3.connect('db_test.db') ## starting the connection to our DB, if not exists, then it creates a new one. 
cursor = conn.cursor() ## it makes all sql staff
data.to_sql('card_aproval_source',con=conn,if_exists='replace',index=False) ## store all the data in our data frame in our sql \
## if exists the table card_aproval_source conn replace it
conn.commit() ## commit changes on our DB. For whatever reason it's not necessary on my Jupyter

# Data Cleaning
## Transforming/ Binarizing

In [7]:
from sklearn import preprocessing

In [8]:
#binarizing income where cut-off is the mean
# 0 : below 72,500 - 1 : More than 72499

Income = data["Income"]

cond_list = [Income < 72500, Income >= 72500]
choice_list = [0, 1]

data["Income_cat"] = np.select(cond_list, choice_list)

In [9]:
#Binarizing number of children where cut-off is the mean
# 0 : less than 2 kids - 1: 2 and more kids

Num_Children = data["Num_Children"]

cond_list = [Num_Children < 2, Num_Children >= 2]
choice_list = [0, 1]

data["Num_Children_cat"] = np.select(cond_list, choice_list)

In [10]:
data

Num_Children  Group  Income  Own_Car  Own_Housing  Target  Income_cat  \
0                  4      0   36151        0            0       0           0   
1                  0      0   36095        1            0       0           0   
2                  2      1   62110        1            1       0           0   
3                  0      1   73644        1            0       0           1   
4                  3      0   99146        0            0       1           1   
...              ...    ...     ...      ...          ...     ...         ...   
499995             1      0   43299        1            1       0           0   
499996             8      1   75956        1            1       1           1   
499997             3      0   66476        0            0       0           0   
499998             2      1   74524        0            0       0           1   
499999             4      0   31129        1            0       0           0   

        Num_Children_cat  
0                      1  
1                      0  
2                      1  
3                      0  
4                      1  
...                  ...  
499995                 0  
499996                 1  
499997                 1  
499998                 1  
499999                 1  

[500000 rows x 8 columns]

In [11]:
data.drop(['Num_Children','Income'], axis=1,inplace= True)

In [12]:
data

Group  Own_Car  Own_Housing  Target  Income_cat  Num_Children_cat
0           0        0            0       0           0                 1
1           0        1            0       0           0                 0
2           1        1            1       0           0                 1
3           1        1            0       0           1                 0
4           0        0            0       1           1                 1
...       ...      ...          ...     ...         ...               ...
499995      0        1            1       0           0                 0
499996      1        1            1       1           1                 1
499997      0        0            0       0           0                 1
499998      1        0            0       0           1                 1
499999      0        1            0       0           0                 1

[500000 rows x 6 columns]

In [13]:
###store data frame after data cleanning and manipulation
data.to_sql('card_aproval_source_manipulated',con=conn,if_exists='replace',index=False) ## store all the data in our data frame in our sql \
## if exists the table card_aproval_source conn replace it
conn.commit() ## commit changes on our DB. For whatever reason it's not necessary on my Jupyter

## Splitting the data into train and test

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
X = data.drop(["Target"],axis=1) # axis: {0 or ‘index’, 1 or ‘columns’}, default 0
y = data["Target"]

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=0)
print("Data sucessfully loaded!")

Data sucessfully loaded!


In [37]:
####################storing all df in SQL ######################
X.to_sql('X',con=conn,if_exists='replace',index=False) ## store all the data in our data frame in our sql \
y.to_sql('y',con=conn,if_exists='replace',index=False) ## store all the data in our data frame in our sql \
X_train.to_sql('X_train',con=conn,if_exists='replace',index=False) ## store all the data in our data frame in our sql \
X_test.to_sql('X_test',con=conn,if_exists='replace',index=False) ## store all the data in our data frame in our sql \
y_train.to_sql('y_train',con=conn,if_exists='replace',index=False) ## store all the data in our data frame in our sql \
y_test.to_sql('y_test',con=conn,if_exists='replace',index=False) ## store all the data in our data frame in our sql \
## if exists the table card_aproval_source conn replace it
conn.commit() ## commit changes on our DB. For whatever reason it's not necessary on my Jupyter

## Training a machine learning model

In [24]:
import xgboost
model = xgboost.XGBClassifier().fit(X_train, y_train)
y_test_predict = model.predict(X_test)
y_test_predict

array([1, 0, 0, ..., 0, 0, 1])

In [39]:
#type(model)
type(y_test_predict)

numpy.ndarray

# Bias Detection
## Representation of Sensitive Variables

In [32]:
sensitive = 'Num_Children_cat' # sensible variable one wants to test for biases
labels = {0:'Below 2',1:'2 or More'} # 0 - Less than 2 kids and 1 - 2 or more kids
fd.identify_bias(model,X_test,y_test,sensitive,labels)

REPRESENTATION


╒════╤═══════════╤═════════════╕
│    │   Below 2 │   2 or More │
╞════╪═══════════╪═════════════╡
│  0 │  0.613326 │    0.614226 │
├────┼───────────┼─────────────┤
│  1 │  0.386674 │    0.385774 │
╘════╧═══════════╧═════════════╛ 

Accept H0: No Significant Relation Between Num_Children_cat and Target Detected. p= 0.7790966568927351


ABILITY


Accept H0: True Positive Disparity Not Detected. p= 0.9916472303121274
Accept H0: False Positive Disparity Not Detected. p= 0.9959732691544855
Accept H0: True Negative Disparity Not Detected. p= 0.9989931576545347
Accept H0: False Negative Disparity Not Detected. p= 0.9814200660354369


PREDICTIVE


Accept H0: No Significant Predictive Disparity. p= 0.9986614271240652


In our case study of the synthetic credit card approval dataset, we are provided with five, fact-based features to determine whether a candidate should be approved a credit card. Among this dataset are two main sensitive groups, income, and number of children. For this particular analysis, we will focus on Number of Children as being our sensitive group. To generate predictions, we will use an XGBoost classifier which received an overall test accuracy of (76%) indicating moderate predictive performance.


## Model Importance Comparison

In [ ]:
sensitive = 'Num_Children_cat' # sensible variable one wants to test for biases
labels = {0:'Below 2',1:'2 or More'} # 0 - Less than 2 kids and 1 - 2 or more kids
fd.understand_shap(X_test,y_test,model, labels,'Num_Children_cat',1,0)

In [40]:
conn.close()